# Investigation of seqs showed up in k-seq but not round 5


## Include all sequences showed up in valid rounds

Sequences only showed up in k-seq will also be included

Valid rounds: <br/>
[1, 2, 3, 4, 5, 6, <br/>
 7, 8,<br/>
 13, 14, 15,<br/>
 19, 20, 21, 23]
 
 **Output**<br/>

A **List** of sequences in showed up in round 5 and valid k-seqs with corresponding counts detected in reach k-seq rounds <br/>
[[seq, [round 5, k-seq rounds * 24]],...]

In [ ]:
import pickle
from __future__ import division

def readSeqs(dirc, countCutoff=1, normalize=True, quantFactor=1):
    #read all sequences from dirc and normalized counts on total counts and quantFactor
    #input fileType: a list of sequences followed by count numbers, with three lines of header info
    
    allSeqCounts = {}

    print('Calculating %s ...' %dirc)

    with open(dirc) as f:
     # get uniques and totals from input file heads
        line0 = next(f)
        uniqueSplit = [elem for elem in line0.strip().split()]
        # print(uniqueSplit)
        uniques = float(uniqueSplit[-1])
        line1 = next(f)
        totalSplit = [elem for elem in line1.strip().split()]
        # print(totalSplit)
        totals = float(totalSplit[-1])
        next(f)
        
    # calculate [normalized] counts    
        i = 0
        for lineRead in f:
            line = [elem for elem in lineRead.strip().split()]
            i += 1
            if int(line[1]) >= countCutoff: # count the seq above cutOff
                if normalize:
                    allSeqCounts[line[0]] = float(line[1])/totals/quantFactor
                else:
                    allSeqCounts[line[0]] = int(line[1])

    return (allSeqCounts, uniques, totals)

def countAll(primDirc, otherDirc, primMin=1, otherMin=1, normList=[], rndToCount=[]):
    totalRndNum = len(otherDirc) + 1
    masterCounts = {}
    counts_init = [0]*totalRndNum
    masterUniques = [0]*totalRndNum
    masterTotals = [0]*totalRndNum
    
 # ---------------------- count masterDirc -----------------------
    round0Counts, round0uniques, round0totals = readSeqs(root + primDirc, countCutoff=primMin, normalize=True,
                                                         quantFactor=normList[0])
    masterUniques[0] = round0uniques
    masterTotals[0] = round0totals
    
    for seq in round0Counts:
        masterCounts[seq] = counts_init[:]
        masterCounts[seq][0] += round0Counts[seq]

 # ------------- count otherLoc ---------------------
    for rnd in rndToCount:
        (roundCounts, uniqs, tots) = readSeqs(root + otherDirc[rnd-1], countCutoff=otherMin, normalize=True, quantFactor=normList[rnd])
        masterUniques[rnd] = uniqs
        masterTotals[rnd] = tots
        for seq in roundCounts:
            if seq in masterCounts:
                masterCounts[seq][rnd] += roundCounts[seq]
            else:
                masterCounts[seq] = counts_init[:]
                masterCounts[seq][rnd] += roundCounts[seq]

    return (masterCounts, masterUniques, masterTotals)

#------- main ---------

root = '/mnt/chen-nas/SequenceData/Abe\'s files/counts-aminoacyl/'
r5Dirc = 'R5c-counts.txt'
kSeqList = ['counts-1A.txt','counts-1B.txt','counts-1C.txt','counts-1D.txt','counts-1E.txt','counts-1F.txt',
            'counts-2A.txt','counts-2B.txt','counts-2C.txt','counts-2D.txt','counts-2E.txt','counts-2F.txt',
            'counts-3A.txt','counts-3B.txt','counts-3C.txt','counts-3D.txt','counts-3E.txt','counts-3F.txt',
            'counts-4A.txt','counts-4B.txt','counts-4C.txt','counts-4D.txt','counts-4E.txt','counts-4F.txt']

with open('/home/yuning/Work/ribozyme_pred/data/k_seq/norm_factor_ng.pkl', 'rb') as openFile:
    quantFactor = pickle.load(openFile)
testNorm = [0.005] + [i for j in quantFactor for i in j]

rndToCountClean = [1, 2, 3, 4, 5, 6,
                   7, 8,
                   13, 14, 15,
                   19, 20, 21, 23]

#------- Count R5 and kSeq data------------

counts, uniqs, tots = countAll(r5Dirc, kSeqList, 1, 1, normList = testNorm, rndToCount = rndToCountClean)

countTable = []
for seq in counts:
    countTable.append([seq, counts[seq]])

# Save count table
outDirc = '/home/yuning/Work/ribozyme_pred/data/k_seq/'
with open(outDirc + 'count_table_all_from_cleanRnds.pkl', "wb") as outFile:
    pickle.dump(countTable, outFile)
    print('Count Table saved to %s' %(outDirc + 'count_table_all_from_cleanRnds.pkl'))

In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from collections import Counter

with open('/home/yuning/Work/ribozyme_pred/data/k_seq/count_table_all_from_cleanRnds.pkl', 'rb') as readFile:
    countTable = pickle.load(readFile)

rndToCount = [[1, 2, 3, 4, 5, 6],
             [7, 8],
             [13, 14, 15],
             [19, 20, 21, 23]]

for seq in countTable:
    seq.append('True')
    seq.append(0)
    for rndBatch in rndToCount:
        counter = 0
        for rnd in rndBatch:
            if seq[1][rnd] != 0:
                counter = 1
            else:
                seq[-2] = False
        seq[-1] += counter

# Save labeled_count table
outDirc = '/home/yuning/Work/ribozyme_pred/data/k_seq/'
with open(outDirc + 'labeled_count_table_allRndSeq.pkl', "wb") as outFile:
    pickle.dump(countTable, outFile)
    print('Labeled count table saved to %s' %(outDirc + 'labeled_count_table_allCleanRndSeq.pkl'))


print('Survey across all the sequences from valids rounds:')      

# print the composition of detected in Round 5
countTableRnd0 = [seq for seq in countTable if seq[1][0]!=0]
countTableNoRnd0 = [seq for seq in countTable if seq[1][0]==0]

print('Total unique sequences: %i' %len(countTable))
print('Unique sequences detected in Round 5: %i' %len(countTableRnd0))
print('Unique sequences not detected in Round 5: %i' %len(countTableNoRnd0))

plt.figure()
plt.bar(x=['Detected in Round 5', 'Not detected in Round 5'], 
        height = [len(countTableRnd0), len(countTableNoRnd0)], width=0.4)
plt.show()


# Print the seqs detected in Round 5
print('For the sequences detected in Round 5:')
print('Unique sequences detected in Round 5: %i' %len(countTableRnd0))
print('\t has non-empty profile %i' %len([i for i in countTableRnd0 if i[-1]>0]))
print('\t has full profile %i' %len([i for i in countTableRnd0 if i[-1]==4]))
print('\t has complete profile %i' %len([i for i in countTableRnd0 if i[-2]]))

profileDist = [i[-1] for i in countTableRnd0 if i[-1]>0]

counter = Counter(profileDist).values()
plt.figure()
plt.bar([1, 2, 3, 4], counter, width = 0.5, align='center', tick_label=[1,2,3,4])
plt.xlabel('# of non-zero concentration profile')
plt.ylabel('Sequences')
plt.show()


# Print the seqs not detected in Round 5
print('For the sequences not detected in Round 5:')
print('Unique sequences not detected in Round 5: %i' %len(countTableNoRnd0))
print('\t has non-empty profile %i' %len([i for i in countTableNoRnd0 if i[-1]>0]))
print('\t has full profile %i' %len([i for i in countTableNoRnd0 if i[-1]==4]))
print('\t has complete profile %i' %len([i for i in countTableNoRnd0 if i[-2]]))
    

profileDist = [i[-1] for i in countTableNoRnd0 if i[-1]>0]

counter = Counter(profileDist).values()
plt.figure()
plt.bar([1, 2, 3, 4], counter, width = 0.5, align='center', tick_label=[1,2,3,4])
plt.xlabel('# of non-zero concentration profile')
plt.ylabel('Sequences')
plt.show()
